<a href="https://colab.research.google.com/github/ningxia202109/llm-learn/blob/main/GraphRAG/graphrag_open_llm_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Reference:**

https://microsoft.github.io/graphrag/posts/get_started/
https://github.com/microsoft/graphrag/issues/339
https://github.com/microsoft/graphrag/issues/339#issuecomment-2250094743
https://mer.vin/2024/07/graphrag-ollama/
https://github.com/severian42/GraphRAG-Local-UI
https://www.reddit.com/r/LocalLLaMA/comments/1du6c8d/graphrag_ollama_intelligent_search_of_local_data/



# 1. [Open LLM Server](https://github.com/rushizirpe/open-llm-server/)

To get started, first we need to clone the open-llm-server repository:

## Step 1: Clone the Repository

In [1]:
# Clone the Repository
!git clone https://github.com/rushizirpe/open-llm-server.git

Cloning into 'open-llm-server'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 88 (delta 42), reused 62 (delta 19), pack-reused 0
Receiving objects: 100% (88/88), 17.88 KiB | 2.98 MiB/s, done.
Resolving deltas: 100% (42/42), done.


## Step 2: Install Dependencies

Navigate into the cloned repository directory and install the required dependencies.

In [2]:
%cd open-llm-server

/content/open-llm-server


In [3]:
# Install Dependencies
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 68.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.3/370.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 13.6 

## Step 3: Launch the Server

Run the following code to start the server.

(The server will initiate the model download process, which may take a while (~5 minutes) if the model isn't already downloaded.)

In [4]:

import subprocess, time
from ipywidgets import HTML
from IPython.display import display

t = HTML(value="0 Seconds", description='Server is Starting Up... Elapsed Time:', style={'description_width': 'initial'})
display(t)

try: subprocess.check_output(['curl', "localhost:1234"])
except: get_ipython().system_raw('uvicorn app:app --host 0.0.0.0 --port 1234 > server.log 2>&1 &')

for timer in range(600):
    try: subprocess.check_output(['curl', "localhost:1234"]); break
    except: time.sleep(1); t.value = f"{timer + 1} Seconds"
else: print("Error: timed out! took more than 10 minutes :(")
print(subprocess.check_output(['curl', "localhost:1234"]))


HTML(value='0 Seconds', description='Server is Starting Up... Elapsed Time:', style=DescriptionStyle(descripti…

b'{"status":"System Status: Operational","gpu":"Available","gpu_details":{"GPU 0":{"compute_capability":[7,5],"device_name":"Tesla T4"}}}'


## Step 4: Verify Connection

To ensure that the server is running correctly, test the API with a sample request:

In [ ]:
!curl http://localhost:1234/

{"status":"System Status: Operational","gpu":"Available","gpu_details":{"GPU 0":{"compute_capability":[7,5],"device_name":"Tesla T4"}}}

### Troubleshooting
If you encounter connection issues, refer to the following:

-  `curl: (7) Failed to connect to localhost port 1234 after 0 ms: Connection refused`
    - Restart the API server [here](#scrollTo=AHOHezZotCHy).
- `curl: (52) Empty reply from server`
    - RAM is insuffiecint to load the LLM model defined in `.yaml`

This completes the setup and initial run of the open-llm-server. You should now have a running server and can interact with it using API requests as we need in graphrag.

# 2. [GraphRAG](https://github.com/microsoft/graphrag/)

This will guide you through the setup and usage of GraphRAG for indexing and querying text data. We will use a sample input file to demonstrate the process.

## Setup

### Step 1: Install

First, install the GraphRAG package:

In [ ]:
!pip install -q graphrag

### Step 2: Initialization

Create the necessary directory structure and initialize the GraphRAG project:

In [ ]:
%cd /content
!mkdir -p ./ragtest/input

/content


In [ ]:
!python -m graphrag.index --init --root ./ragtest

2024-07-20 10:16:10.076908: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 10:16:10.076980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 10:16:10.078980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 10:16:12.298615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing project at ./ragtest
⠋ GraphRAG Indexer 

You will see some warnings related to CUDA and TensorFlow, but they can be ignored if the initialization completes successfully.

### Step 3: Configuration (YAML)

Create a YAML configuration file for GraphRAG:

In [ ]:
%%writefile ./ragtest/settings.yaml

encoding_model: cl100k_base
skip_workflows: []
llm:
  api_key: ${GROQ_API_KEY}
  type: openai_chat # or azure_openai_chat
  model:  llama3-8b-8192
  model_supports_json: true # recommended if this is available for your model.
  max_tokens: 3000
  # request_timeout: 180.0
  api_base: https://api.groq.com/openai/v1
  # api_version: 2024-02-15-preview
  # organization: <organization_id>
  # deployment_name: <azure_model_deployment_name>
  tokens_per_minute: 6000 # set a leaky bucket throttle
  requests_per_minute: 2 # set a leaky bucket throttle
  max_retries: 3
  # max_retry_wait: 10.0
  # sleep_on_rate_limit_recommendation: true # whether to sleep when azure suggests wait-times
  # concurrent_requests: 25 # the number of parallel inflight requests that may be made

parallelization:
  stagger: 0.3
  # num_threads: 50 # the number of threads to use for parallel processing

async_mode: threaded # or asyncio

embeddings:
  ## parallelization: override the global parallelization settings for embeddings
  async_mode: threaded # or asyncio
  llm:
    api_key: ${GRAPHRAG_API_KEY}
    # type: openai_embedding # or azure_openai_embedding
    model: nomic-ai/nomic-embed-text-v1.5
    api_base: http://localhost:1234/v1
    # api_version: 2024-02-15-preview
    # organization: <organization_id>
    # deployment_name: <azure_model_deployment_name>
    # tokens_per_minute: 150_000 # set a leaky bucket throttle
    # requests_per_minute: 10_000 # set a leaky bucket throttle
    # max_retries: 10
    # max_retry_wait: 10.0
    # sleep_on_rate_limit_recommendation: true # whether to sleep when azure suggests wait-times
    # concurrent_requests: 25 # the number of parallel inflight requests that may be made
    # batch_size: 16 # the number of documents to send in a single request
    # batch_max_tokens: 8191 # the maximum number of tokens to send in a single request
    # target: required # or optional



chunks:
  size: 300
  overlap: 100
  group_by_columns: [id] # by default, we don't allow chunks to cross documents

input:
  type: file # or blob
  file_type: text # or csv
  base_dir: "input"
  file_encoding: utf-8
  file_pattern: ".*\\.txt$"

cache:
  type: file # or blob
  base_dir: "cache"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

storage:
  type: file # or blob
  base_dir: "output/${timestamp}/artifacts"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

reporting:
  type: file # or console, blob
  base_dir: "output/${timestamp}/reports"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

entity_extraction:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/entity_extraction.txt"
  entity_types: [organization,person,geo,event]
  max_gleanings: 0

summarize_descriptions:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/summarize_descriptions.txt"
  max_length: 500

claim_extraction:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  # enabled: true
  prompt: "prompts/claim_extraction.txt"
  description: "Any claims or facts that could be relevant to information discovery."
  max_gleanings: 0

community_report:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/community_report.txt"
  max_length: 2000
  max_input_length: 8000

cluster_graph:
  max_cluster_size: 10

embed_graph:
  enabled: false # if true, will generate node2vec embeddings for nodes
  # num_walks: 10
  # walk_length: 40
  # window_size: 2
  # iterations: 3
  # random_seed: 597832

umap:
  enabled: false # if true, will generate UMAP embeddings for nodes

snapshots:
  graphml: false
  raw_entities: false
  top_level_nodes: false

local_search:
  # text_unit_prop: 0.5
  # community_prop: 0.1
  # conversation_history_max_turns: 5
  # top_k_mapped_entities: 10
  # top_k_relationships: 10
  # max_tokens: 12000

global_search:
  # max_tokens: 12000
  # data_max_tokens: 12000
  # map_max_tokens: 1000
  # reduce_max_tokens: 2000
  # concurrency: 32

Overwriting ./ragtest/settings.yaml


### Step 4: Sample Input

Create a sample input text file:

In [ ]:
%%writefile /content/ragtest/input/sample.txt

MARLEY'S GHOST

Marley was dead, to begin with. There is no doubt whatever about that.
The register of his burial was signed by the clergyman, the clerk, the
undertaker, and the chief mourner. Scrooge signed it. And Scrooge's name
was good upon 'Change for anything he chose to put his hand to. Old
Marley was as dead as a door-nail.


Writing /content/ragtest/input/sample.txt


### Step 5: Environment Variables

Set the necessary environment variables for API keys.
- For models with higher parameters, the GROQ_API_KEY is essential since they require high-end GPUs. While the GROQ API is free, it comes with certain limitations.
- Additionally, the HUGGING_FACE_TOKEN is needed if you're using LLMs from gated repositories on Hugging Face.

In [ ]:

import os
GROQ_API_KEY = '<GROQ_API_KEY>' # @param {type:"string"}
HUGGING_FACE_TOKEN = '<HUGGING_FACE_TOKEN>' # @param {type:"string"}

# Get your API Key from https://console.groq.com/keys
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ['GRAPHRAG_API_KEY'] = 'DUMMY_KEY'
# If gated LLM
os.environ['HUGGING_FACE_TOKEN'] = HUGGING_FACE_TOKEN

if len(os.getenv("GROQ_API_KEY"))<25:
    assert False, "GROQ_API_KEY is required. Sign up and Get your API Key from https://console.groq.com/keys"
# print(os.getenv('REDDIT_NAME'))

## Indexing

Run the indexing process to analyze the input text:

- The indexing process will output the progress of various workflows, and you should see a message indicating that all workflows completed successfully.

In [ ]:
!python -m graphrag.index --root ./ragtest

2024-07-20 10:18:24.194536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 10:18:24.194604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 10:18:24.196287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 10:18:25.872579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
🚀 Reading settings from ragtest/settings.yaml
⠧ GraphRAG Indexer 
⠧ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
⠧ GraphRAG Indexer 
├── Loading Input (text) - 1 files

## Query

Perform a query on the indexed data:

- The query response should provide insights about Marley based on the input text, including information about Marley's significance and legacy.

### Global Search

In [ ]:
!python -m graphrag.query --root ./ragtest --method global "Who is Marley?"

2024-07-20 10:19:23.422270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 10:19:23.422331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 10:19:23.423734: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 10:19:24.996659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from ragtest/settings.yaml
creating llm client with {'api_key': 'REDACTED,len=56', 'type': "openai_chat", 'model': 'llama3-8b-8192', 'max_tokens': 3000, 'request_timeout': 180.0, 'api_base': 'https://api.groq.com/ope

### Local Search

In [ ]:
!python -m graphrag.query --root ./ragtest --method local "Who is Marley?"

2024-07-20 14:47:16.856466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 14:47:16.856516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 14:47:16.857700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 14:47:18.014055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from ragtest/settings.yaml
[2024-07-20T14:47:26Z WARN  lance::dataset] No existing dataset at /content/lancedb/description_embedding.lance, it will be created
creating llm client with {'api_key': 'REDACTED,len=56', '


If you like this usage of project, please consider starring it on GitHub!

<div align="center">
    <a href="https://github.com/rushizirpe/open-llm-server" target="_blank">
        <img src="https://img.shields.io/github/stars/rushizirpe/open-llm-server?style=social" alt="GitHub stars">
    </a>
</div>

## With Local Model [OPTIONAL]
This configuration is suitable for systems with high-end GPUs and RAM, such as the A100.


In [ ]:
import torch

def get_gpu_name():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        return gpu_name
    else:
        return None

gpu_name = get_gpu_name()

if gpu_name:
    print(f"GPU Name: {gpu_name}")
    if "A100" in gpu_name:
        print("A100 GPU detected. Advance Configuration can be used.")
    else:
        assert True, "Non-A100 GPU detected."
else:
    assert True, "No GPU detected."


In [ ]:
%%writefile ./ragtest/settings.yaml

encoding_model: cl100k_base
skip_workflows: []
llm:
  # api_key: ${GROQ_API_KEY}
  api_key: DUMMY_KEY
  # type: openai_chat # or azure_openai_chat
  model:  llama3-8b-8192
  model_supports_json: true # recommended if this is available for your model.
  max_tokens: 3000
  # request_timeout: 180.0
  api_base: http://localhost:1234/v1
  # api_version: 2024-02-15-preview
  # organization: <organization_id>
  # deployment_name: <azure_model_deployment_name>
  # tokens_per_minute: 6000 # set a leaky bucket throttle
  # requests_per_minute: 2 # set a leaky bucket throttle
  # max_retries: 3
  # max_retry_wait: 10.0
  # sleep_on_rate_limit_recommendation: true # whether to sleep when azure suggests wait-times
  # concurrent_requests: 25 # the number of parallel inflight requests that may be made

parallelization:
  stagger: 0.3
  # num_threads: 50 # the number of threads to use for parallel processing

async_mode: threaded # or asyncio

embeddings:
  ## parallelization: override the global parallelization settings for embeddings
  async_mode: threaded # or asyncio
  llm:
    # api_key: ${GRAPHRAG_API_KEY}
    api_key: DUMMY_KEY
    # type: openai_embedding # or azure_openai_embedding
    model: nomic-ai/nomic-embed-text-v1.5
    api_base: http://localhost:1234/v1
    # api_version: 2024-02-15-preview
    # organization: <organization_id>
    # deployment_name: <azure_model_deployment_name>
    # tokens_per_minute: 150_000 # set a leaky bucket throttle
    # requests_per_minute: 10_000 # set a leaky bucket throttle
    # max_retries: 10
    # max_retry_wait: 10.0
    # sleep_on_rate_limit_recommendation: true # whether to sleep when azure suggests wait-times
    # concurrent_requests: 25 # the number of parallel inflight requests that may be made
    # batch_size: 16 # the number of documents to send in a single request
    # batch_max_tokens: 8191 # the maximum number of tokens to send in a single request
    # target: required # or optional



chunks:
  size: 300
  overlap: 100
  group_by_columns: [id] # by default, we don't allow chunks to cross documents

input:
  type: file # or blob
  file_type: text # or csv
  base_dir: "input"
  file_encoding: utf-8
  file_pattern: ".*\\.txt$"

cache:
  type: file # or blob
  base_dir: "cache"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

storage:
  type: file # or blob
  base_dir: "output/${timestamp}/artifacts"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

reporting:
  type: file # or console, blob
  base_dir: "output/${timestamp}/reports"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

entity_extraction:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/entity_extraction.txt"
  entity_types: [organization,person,geo,event]
  max_gleanings: 0

summarize_descriptions:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/summarize_descriptions.txt"
  max_length: 500

claim_extraction:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  # enabled: true
  prompt: "prompts/claim_extraction.txt"
  description: "Any claims or facts that could be relevant to information discovery."
  max_gleanings: 0

community_report:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/community_report.txt"
  max_length: 2000
  max_input_length: 8000

cluster_graph:
  max_cluster_size: 10

embed_graph:
  enabled: false # if true, will generate node2vec embeddings for nodes
  # num_walks: 10
  # walk_length: 40
  # window_size: 2
  # iterations: 3
  # random_seed: 597832

umap:
  enabled: false # if true, will generate UMAP embeddings for nodes

snapshots:
  graphml: false
  raw_entities: false
  top_level_nodes: false

local_search:
  # text_unit_prop: 0.5
  # community_prop: 0.1
  # conversation_history_max_turns: 5
  # top_k_mapped_entities: 10
  # top_k_relationships: 10
  # max_tokens: 12000

global_search:
  # max_tokens: 12000
  # data_max_tokens: 12000
  # map_max_tokens: 1000
  # reduce_max_tokens: 2000
  # concurrency: 32

Overwriting ./ragtest/settings.yaml


In [ ]:
!python -m graphrag.index --root ./ragtest

2024-07-20 05:31:18.379828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 05:31:18.379881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 05:31:18.381046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 05:31:19.561703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
🚀 Reading settings from ragtest/settings.yaml
⠴ GraphRAG Indexer 
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files

In [ ]:
!python -m graphrag.query --root ./ragtest --method global "Who is Marley?"

2024-07-20 05:32:55.863967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 05:32:55.864017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 05:32:55.865295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 05:32:57.045486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from ragtest/settings.yaml
creating llm client with {'api_key': 'REDACTED,len=56', 'type': "openai_chat", 'model': 'llama3-8b-8192', 'max_tokens': 3000, 'request_timeout': 180.0, 'api_base': 'https://api.groq.com/ope

In [ ]:
!python -m graphrag.query --root ./ragtest --method local "Who is Marley?"

2024-07-20 14:47:16.856466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 14:47:16.856516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 14:47:16.857700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 14:47:18.014055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from ragtest/settings.yaml
[2024-07-20T14:47:26Z WARN  lance::dataset] No existing dataset at /content/lancedb/description_embedding.lance, it will be created
creating llm client with {'api_key': 'REDACTED,len=56', '